In [1]:
import pandas as pd
import numpy as np

inputfile = './chapter5/demo/data/DMCT_menu_orders.xls'

data = pd.read_excel(inputfile, header=None)

In [4]:
def get_C1(data: pd.DataFrame):
    
    C1 = {}
    for col in data:
        for i in data[col]:
            C1.setdefault(i, 0)
            C1[i] = C1[i] + 1

    if np.nan in C1:
        del C1[np.nan]
        
    C1 = pd.DataFrame({'itemset': [set([s]) for s in list(C1.keys())], 'count': list(C1.values())}) #注意这一步将dict的key转化为set的做法
    
    return C1

In [6]:
def trim_C(C, min_sup):
    
    L =  C[C['count'] >= min_sup]
    L = L[['itemset','count']]
    return L

import itertools

def findsubsets(s,m):
    return set(itertools.combinations(s, m))


def _connect(L: pd.DataFrame):
    pre_C  = []
    Lkeys = list(L.itemset)
    
    # 需要限制可以进行连接的情况
    lengthOfL = len(Lkeys)
    for i in range(lengthOfL-1):
        listed_i = list(Lkeys[i])
        listed_i.sort()
        for j in range(i+1, lengthOfL):
            listed_j = list(Lkeys[j])
            listed_j.sort()
            if listed_i[:-1] == listed_j[:-1]:
                pre_C.append(Lkeys[i] | Lkeys[j])
    return pre_C


def _move_candidate_has_infrequent_subset(pre_C: list, k, L_: dict):
    for s in pre_C: 
        for subset in findsubsets(s, k-1):
            if set(subset) not in list(L_[k-1].itemset):
                if pre_C != []: #防止pre_C已空的情况下,继续remove报错
                    pre_C.remove(s)
    return pre_C


def _count_support(pre_C, D):
    # 创建C的骨架(dataframe)
    C = pd.DataFrame(columns=['itemset', 'count'])
    for i in pre_C:
        C = C.append([{'itemset':i, 'count':0}], ignore_index=True)#非in-place操作,注意赋值回去给C
    
    for index, row in D.iterrows():
        row = list(row)
        for cb in pre_C:
            if cb <= set(row):
                C.loc[C.itemset==cb, 'count'] += 1
    return C


# 由L1生成C2
#apriori_gen
def L2C(L: pd.DataFrame, D: pd.DataFrame, k: int, L_: list):
    
    # 1. 连接
    pre_C = _connect(L)

    #2. 剪枝, 删除非频繁候选
    pre_C = _move_candidate_has_infrequent_subset(pre_C, k, L_)
     
    #3. 支持度计数
    C= _count_support(pre_C, D)
    
    return C


    

In [177]:
def confidence(L: pd.DataFrame, L_):
    itemset_ses = L['itemset']
    k = len(itemset_ses[0])
    conf_df = pd.DataFrame()
    for itemset in itemset_ses:        
        for ik in range(1, k):
            subsets = findsubsets(itemset, ik)
            for subset in subsets:
                subset = set(subset)
#                 print('itemset', itemset)
#                 print('subset', subset)
                diffset = itemset - subset #求出差集
                c_itemset = list(L[L['itemset'] == itemset]['count'])[0]
                _L = L_[ik]
#                 print('_L', _L)
                c_subset = list(_L[_L['itemset'] == subset]['count'])[0]
                conf = c_itemset / c_subset
                conf_df= conf_df.append({'itemset': itemset, 'start': subset, 'subset_count': c_subset,  'end': diffset, 'itemset_count': c_itemset, 'conf': conf}, ignore_index=True)
    conf_df = conf_df[['itemset', 'start','end', 'subset_count',  'itemset_count', 'conf']]
    return conf_df
                

In [178]:
C1 = get_C1(data)
L1 = trim_C(C1,2)
#L1 = L1[['itemset','count']] #调整L1列的顺序, 顺序调整: count itemset -> itemset count
L_ = {1: L1}
C_ = {1: C1}
k = 1

while not L_[k].empty:
    #print(L_[k])
    C_[k+1] = L2C(L_[k], data, k+1, L_)
    L_[k+1] = trim_C(C_[k+1], 2)
    k += 1

In [179]:
from IPython.display import display
for i in L_:
    display(L_[i])
    #print(L_[i].to_html())

,itemset,count
0,{I1},6
1,{I2},7
2,{I4},2
3,{I3},6
4,{I5},2


,itemset,count
0,"{I2, I1}",4
2,"{I1, I3}",4
3,"{I5, I1}",2
4,"{I2, I4}",2
5,"{I2, I3}",4
6,"{I2, I5}",2


,itemset,count
0,"{I2, I1, I3}",2
1,"{I2, I5, I1}",2


,itemset,count


In [180]:
L2 = L_[2]

In [181]:
L2 = L_[2]

In [182]:
L2

,itemset,count
0,"{I2, I1}",4
2,"{I1, I3}",4
3,"{I5, I1}",2
4,"{I2, I4}",2
5,"{I2, I3}",4
6,"{I2, I5}",2


In [183]:
confidence(L2, L_)

,itemset,start,end,subset_count,itemset_count,conf
0,"{I2, I1}",{I2},{I1},7.0,4.0,0.571429
1,"{I2, I1}",{I1},{I2},6.0,4.0,0.666667
2,"{I1, I3}",{I3},{I1},6.0,4.0,0.666667
3,"{I1, I3}",{I1},{I3},6.0,4.0,0.666667
4,"{I5, I1}",{I1},{I5},6.0,2.0,0.333333
5,"{I5, I1}",{I5},{I1},2.0,2.0,1.000000
6,"{I2, I4}",{I4},{I2},2.0,2.0,1.000000
7,"{I2, I4}",{I2},{I4},7.0,2.0,0.285714
8,"{I2, I3}",{I3},{I2},6.0,4.0,0.666667
9,"{I2, I3}",{I2},{I3},7.0,4.0,0.571429


In [185]:
L3 = L_[3]
confidence(L3, L_).sort_values(by=['conf'], ascending=False)

,itemset,start,end,subset_count,itemset_count,conf
8,"{I2, I5, I1}",{I5},"{I2, I1}",2.0,2.0,1.000000
10,"{I2, I5, I1}","{I1, I5}",{I2},2.0,2.0,1.000000
11,"{I2, I5, I1}","{I2, I5}",{I1},2.0,2.0,1.000000
3,"{I2, I1, I3}","{I2, I3}",{I1},4.0,2.0,0.500000
4,"{I2, I1, I3}","{I1, I3}",{I2},4.0,2.0,0.500000
5,"{I2, I1, I3}","{I2, I1}",{I3},4.0,2.0,0.500000
9,"{I2, I5, I1}","{I2, I1}",{I5},4.0,2.0,0.500000
0,"{I2, I1, I3}",{I3},"{I2, I1}",6.0,2.0,0.333333
2,"{I2, I1, I3}",{I1},"{I2, I3}",6.0,2.0,0.333333
6,"{I2, I5, I1}",{I1},"{I2, I5}",6.0,2.0,0.333333


In [192]:
df = confidence(L3, L_)
df[df['itemset'] == set(['I1','I2','I5'])]

,itemset,start,end,subset_count,itemset_count,conf
6,"{I2, I5, I1}",{I1},"{I2, I5}",6.0,2.0,0.333333
7,"{I2, I5, I1}",{I2},"{I1, I5}",7.0,2.0,0.285714
8,"{I2, I5, I1}",{I5},"{I2, I1}",2.0,2.0,1.000000
9,"{I2, I5, I1}","{I2, I1}",{I5},4.0,2.0,0.500000
10,"{I2, I5, I1}","{I1, I5}",{I2},2.0,2.0,1.000000
11,"{I2, I5, I1}","{I2, I5}",{I1},2.0,2.0,1.000000


In [186]:
a = L2[L2['itemset'] == set(['I1', 'I3'])]['count']

In [187]:
list(a)

[4]

In [189]:
for i in L_[2]['itemset']:
    display(i)

{'I1', 'I2'}

{'I1', 'I3'}

{'I1', 'I5'}

{'I2', 'I4'}

{'I2', 'I3'}

{'I2', 'I5'}